In [2]:
import numpy as np
import pandas as pd
import datetime

In [3]:
# Target Group: adult patients' first icu visit
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)

# Create ICU Id list
icu = []
for icuid in df['icustay_id']:
    icu.append(icuid)

# Max chloride
df = pd.read_csv('chloride.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.filter(['subject_id','hadm_id','icustay_id','icu_day','chloride_max','chloride_input_meq'])
df= df.query('icu_day == 1')
# df = df.dropna(subset = ['chloride_max'])

df_chl_max = df.filter(['subject_id','hadm_id','icustay_id','chloride_max'])
df_chl_max = df_chl_max.dropna(subset = ['chloride_max'])
df_chl_input = df.filter(['subject_id','hadm_id','icustay_id','chloride_input_meq'])

df_chl_max = df_chl_max.set_index(['subject_id','hadm_id','icustay_id'])
df_chl_input = df_chl_input.set_index(['subject_id','hadm_id','icustay_id'])

In [4]:
# Target Group: adult patients' first icu visit
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)

# Create ICU Id list
icu = []
for icuid in df['icustay_id']:
    icu.append(icuid)

    
# Max chloride
df = pd.read_csv('chloride.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.filter(['subject_id','hadm_id','icustay_id','icu_day','chloride_max','chloride_input_meq'])
df= df.query('icu_day == 1')
# df = df.dropna(subset = ['chloride_max'])

df_chl_max = df.filter(['subject_id','hadm_id','icustay_id','chloride_max'])
df_chl_max = df_chl_max.dropna(subset = ['chloride_max'])
df_chl_input = df.filter(['subject_id','hadm_id','icustay_id','chloride_input_meq'])


df_chl_max = df_chl_max.set_index(['subject_id','hadm_id','icustay_id'])
df_chl_input = df_chl_input.set_index(['subject_id','hadm_id','icustay_id'])

# Max chloride
#df = pd.read_csv('chloride.csv')
#df = df[df['icustay_id'].isin(icu)]
#df = df.set_index(['subject_id','hadm_id','icustay_id'])
#df = df.filter(['icustay_id','icu_day','chloride_max','chloride_input_meq'])
# df = df.dropna(subset = ['chloride_max'])

# Chloride >= 110 in the second day
#df_chl = df.query('icu_day == 2')
#df_chl['chl_110'] = (df_chl['chloride_max'] >= 110).astype(int)

# Demographic
df = pd.read_csv('adm_demographics.csv')
df = df.set_index(['subject_id', 'hadm_id'])
df_demo = df.filter(['insurance','ethnicity','age','gender'])

# Weight
df = pd.read_csv('weight.csv')
df = df[df['icustay_id'].isin(icu)]
df_wt = df.filter(['icustay_id','day','weight'])

# GCS(Glasgow coma scale)
df = pd.read_csv('gcs_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df_gcs = df.filter(['icustay_id','day','mingcs'])

# First-day Vitals(Merge with GCS and Weight)
df = pd.read_csv('vitals_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id','icustay_id'])
df_vitals = df.filter(['day', 'heartrate_max','sysbp_min','diasbp_min','resprate_max','spo2_min','tempc_max'])
df_vitals = df_vitals.reset_index()
df_vitals = df_vitals.merge(df_wt, on = ['icustay_id','day']).merge(df_gcs, on = ['icustay_id','day'])
df_vitals = df_vitals.query('(day == 1) & (0 < heartrate_max <= 250) &(40<sysbp_min <190) &(20<diasbp_min<150) &(6<resprate_max<80) &(0<spo2_min<100)&(30<tempc_max<44)&(30<weight<500)&(3<mingcs<15)'
                           )
df_vitals = df_vitals.set_index(['subject_id','hadm_id','icustay_id'])

# Medication from prescription table
df = pd.read_csv('medication.csv')
df_med = df.drop(df.columns[[0,3]], axis=1)
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)
df = df_med.merge(df, on = ['subject_id','hadm_id'])
df['taken'] = ((pd.to_datetime(df['intime']) <= pd.to_datetime(df['startdate'])) & 
               ((pd.to_datetime(df['startdate']) <= (pd.to_datetime(df['intime']) + datetime.timedelta(days=1)))))
df = df.filter(['subject_id','hadm_id','icustay_id','drug','taken'])
df_med = df.set_index(['subject_id','hadm_id','icustay_id'])
df_med = df_med.groupby(['subject_id','hadm_id','icustay_id','drug']).any()
df_med = df_med['taken'].unstack()
df_med = (df_med * 1).fillna(0)

# Intake and output
df = pd.read_csv('fluid_io.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.query('icu_day == 1')
df_fluid = df.set_index(['subject_id','hadm_id','icustay_id'])
df_fluid.head()

# Diagnosis Code(group by elixhauser)
df = pd.read_csv('elixhauser.csv')
df_icd = df.drop(df.columns[[0]], axis=1)
df_icd = df_icd.set_index('hadm_id')

# Labs
df = pd.read_csv('labs1.csv')
df = df.drop(df.columns[0], axis=1)
df = df[df['icustay_id'].isin(icu)]
df_labs = df.query('day == 1 & 0.1<lactate_max<40 & 90<sodium_max<190 & 1.5<potassium_max<15 & 1<albumin_max<6.5 & 5<bicarbonate_min<65 & 0.1<creatinine_max<47 & 66<chloride_max<170 & 18<glucose_max<2500 & 2<hemoglobin_min<21 & 5<platelet_min<1600  & 0.5<inr_max<20 & 1<bun_max<280 & 0.1<wbc_max<680 & 0.1<bilirubin_min<80 & 1.5<calcium_total_min<18 & 0.2<calcium_ionized_min<2.5 & 2<ast_sgot_max<25000 & 3<amylase_max<25000 &  1<lipase_max<25000 & 0.1<c_reactive_protein_max<300')
df_labs = df_labs.set_index(['subject_id','hadm_id','icustay_id'])

# Interventions
df = pd.read_csv('interventions.csv')
df = df.query('day == 1')
df_int = df.drop(df.columns[[0]], axis=1)
df_int = df_int.set_index(['subject_id', 'hadm_id', 'icustay_id'])
df_int = df_int.filter(['label'])
df_int['taken'] = 1
df_int = df_int.groupby(['subject_id','hadm_id','icustay_id','label']).any()
df_int = df_int['taken'].unstack()
df_int = (df_int * 1).fillna(0)

In [5]:
demo = ['age', 'gender', 'ethnicity']
vitals = ['heartrate_max', 'sysbp_min', 'diasbp_min', 'resprate_max', 'weight', 'mingcs']
medication = ['norepinephrine']
fluid = ['fluid_net_input_ml']
diag = ['congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 
        'chronic_pulmonary', 
       'diabetes_complicated',  'renal_failure',
       'liver_disease',  
        'coagulopathy',  'fluid_electrolyte',
       'blood_loss_anemia']
labs = [ #'chloride_max',
    'sodium_max', 'bicarbonate_min']
interventions = ['EPAP','IPAP','LPM','MeanAirwayPressure']

In [6]:
df_demo = df_demo[demo]
df_vitals = df_vitals[vitals]
df_med = df_med[medication]
df_fluid = df_fluid[fluid]
df_icd = df_icd[diag]
df_labs = df_labs[labs]
df_int = df_int[interventions]
#df_chl = df_chl[['chl_110','chloride_input_meq']]

In [7]:
df = df_chl_max.join(df_demo, how = 'inner') \
.join(df_chl_input, how = 'left') \
.join(df_fluid, how = 'left') \
.join(df_icd, how = 'left') \
.join(df_labs, how = 'left') \
.join(df_int, how='left') \
.join(df_vitals, how='left') \
.join(df_med, how = 'left') 

df = df.fillna(0)

chloride_max        age gender ethnicity  \
subject_id hadm_id icustay_id                                             
3          145834  211552             114.0  76.575342      M     WHITE   
4          185777  294638             108.0  47.876712      F     WHITE   
6          107064  228232             101.0  65.983562      F     WHITE   
9          150750  220597             103.0  41.816438      M     OTHER   
11         194540  229441             106.0  50.180822      F     WHITE   
12         112213  232669             113.0  72.419178      M     WHITE   
13         143045  263738             116.0  39.890411      F     WHITE   
17         161087  257980             113.0  47.849315      F     WHITE   
           194023  277042             112.0  47.484932      F     WHITE   
18         188822  298129             102.0  50.873973      M     WHITE   
19         109235  273430             105.0  91.400000      M     WHITE   
20         157681  264490             108.0  75.926027      F     WHITE   
21         109451  217847             104.0  87.495890      M     WHITE   
           111970  216859             104.0  87.882192      M     WHITE   
22         165315  204798             105.0  64.969863      F     WHITE   
23         124321  234044             108.0  75.304110      M     WHITE   
           152223  227807             114.0  71.178082      M     WHITE   
24         161859  262236             107.0  39.041096      M     WHITE   
25         129635  203487             111.0  58.989041      M     WHITE   
26         197661  244882             106.0  72.052055      M     OTHER   
28         162569  225559             112.0  74.432877      M     WHITE   
30         104557  225176              97.0  91.400000      M     OTHER   
31         128652  254478             103.0  72.312329      M     WHITE   
32         175413  295037             104.0  45.542466      M     WHITE   
33         176176  296681             106.0  82.446575      M     OTHER   
34         115799  263086             105.0  91.400000      M     WHITE   
           144319  290505             107.0  91.400000      M     WHITE   
35         166707  282039             120.0  69.487671      F     WHITE   
36         122659  211200             109.0  69.778082      M     WHITE   
           165660  241249             109.0  72.775342      M     WHITE   
...                                     ...        ...    ...       ...   
99897      181057  298720             101.0  54.493151      M     BLACK   
99899      188409  245756              92.0  87.013699      M     BLACK   
99901      131711  226541              86.0  68.873973      M     WHITE   
99903      135939  291066             109.0  50.142466      M     BLACK   
99912      189380  236183             112.0  84.600000      M     WHITE   
99913      175989  292375             112.0  71.136986      M  HISPANIC   
99922      123563  259919             112.0  69.326027      M     WHITE   
99923      164914  202505              92.0  54.424658      M     WHITE   
           192053  298857             108.0  54.646575      M     WHITE   
99928      104802  236704             104.0  20.884932      F  HISPANIC   
99936      107913  213906             107.0  91.400000      F     WHITE   
99937      129380  291004             103.0  65.720548      F  HISPANIC   
99938      103815  296707             107.0  80.890411      M     WHITE   
99939      159023  252442             107.0  26.967123      M  HISPANIC   
99944      185654  221067             106.0  85.536986      F     WHITE   
99946      157197  298613             105.0  49.698630      M     WHITE   
99955      108494  243255             115.0  63.232877      M     BLACK   
99957      148574  257127              98.0  68.095890      M     WHITE   
99965      101083  257338             108.0  66.334247      M     WHITE   
99966      167228  252173             106.0  77.536986      M     WHITE   
99973      150202  275083             117.0  65.542466  

In [8]:
df['gender'] = (df['gender'] == 'F').astype(int)

In [9]:
df = pd.concat([df, pd.get_dummies(df['ethnicity'])], axis=1)
df = df.drop(['ethnicity', 'OTHER'], axis=1)

chloride_max        age  gender  \
subject_id hadm_id icustay_id                                    
3          145834  211552             114.0  76.575342       0   
4          185777  294638             108.0  47.876712       1   
6          107064  228232             101.0  65.983562       1   
9          150750  220597             103.0  41.816438       0   
11         194540  229441             106.0  50.180822       1   
12         112213  232669             113.0  72.419178       0   
13         143045  263738             116.0  39.890411       1   
17         161087  257980             113.0  47.849315       1   
           194023  277042             112.0  47.484932       1   
18         188822  298129             102.0  50.873973       0   
19         109235  273430             105.0  91.400000       0   
20         157681  264490             108.0  75.926027       1   
21         109451  217847             104.0  87.495890       0   
           111970  216859             104.0  87.882192       0   
22         165315  204798             105.0  64.969863       1   
23         124321  234044             108.0  75.304110       0   
           152223  227807             114.0  71.178082       0   
24         161859  262236             107.0  39.041096       0   
25         129635  203487             111.0  58.989041       0   
26         197661  244882             106.0  72.052055       0   
28         162569  225559             112.0  74.432877       0   
30         104557  225176              97.0  91.400000       0   
31         128652  254478             103.0  72.312329       0   
32         175413  295037             104.0  45.542466       0   
33         176176  296681             106.0  82.446575       0   
34         115799  263086             105.0  91.400000       0   
           144319  290505             107.0  91.400000       0   
35         166707  282039             120.0  69.487671       1   
36         122659  211200             109.0  69.778082       0   
           165660  241249             109.0  72.775342       0   
...                                     ...        ...     ...   
99897      181057  298720             101.0  54.493151       0   
99899      188409  245756              92.0  87.013699       0   
99901      131711  226541              86.0  68.873973       0   
99903      135939  291066             109.0  50.142466       0   
99912      189380  236183             112.0  84.600000       0   
99913      175989  292375             112.0  71.136986       0   
99922      123563  259919             112.0  69.326027       0   
99923      164914  202505              92.0  54.424658       0   
           192053  298857             108.0  54.646575       0   
99928      104802  236704             104.0  20.884932       1   
99936      107913  213906             107.0  91.400000       1   
99937      129380  291004             103.0  65.720548       1   
99938      103815  296707             107.0  80.890411       0   
99939      159023  252442             107.0  26.967123       0   
99944      185654  221067             106.0  85.536986       1   
99946      157197  298613             105.0  49.698630       0   
99955      108494  243255             115.0  63.232877       0   
99957      148574  257127              98.0  68.095890       0   
99965      101083  257338             108.0  66.334247       0   
99966      167228  252173             106.0  77.536986       0   
99973      150202  275083             117.0  65.542466       1   
99982      112748  275958              99.0  65.304110       0   
           151454  221194             100.0  65.200000       0   
           183791  239827              92.0  65.419178       0   
99983      117390  286606             106.0  78.627397       0   
99985      176670  279638             105.0  53.843836       0   
99991      151118  226241             114.0  47.747945       0   
99992      197084  242052             104.0  65.813699       1   
99995

In [10]:
#filter the hyperchloremic patients

df = df.query('chloride_max < 110') 

chloride_max        age  gender  \
subject_id hadm_id icustay_id                                    
4          185777  294638             108.0  47.876712       1   
6          107064  228232             101.0  65.983562       1   
9          150750  220597             103.0  41.816438       0   
11         194540  229441             106.0  50.180822       1   
18         188822  298129             102.0  50.873973       0   
19         109235  273430             105.0  91.400000       0   
20         157681  264490             108.0  75.926027       1   
21         109451  217847             104.0  87.495890       0   
           111970  216859             104.0  87.882192       0   
22         165315  204798             105.0  64.969863       1   
23         124321  234044             108.0  75.304110       0   
24         161859  262236             107.0  39.041096       0   
26         197661  244882             106.0  72.052055       0   
30         104557  225176              97.0  91.400000       0   
31         128652  254478             103.0  72.312329       0   
32         175413  295037             104.0  45.542466       0   
33         176176  296681             106.0  82.446575       0   
34         115799  263086             105.0  91.400000       0   
           144319  290505             107.0  91.400000       0   
36         122659  211200             109.0  69.778082       0   
           165660  241249             109.0  72.775342       0   
37         188670  213503             103.0  68.972603       0   
38         185910  248910             107.0  75.991781       0   
41         101757  261027             104.0  56.671233       0   
42         119203  210828             105.0  61.205479       0   
45         157907  274249             109.0  42.432877       0   
46         144073  268016             106.0  74.046575       0   
49         190539  249195             105.0  80.969863       0   
52         190797  261857             101.0  38.147945       0   
53         155385  239839             104.0  45.580822       0   
...                                     ...        ...     ...   
99870      124555  220524             105.0  43.621918       1   
99873      143544  206270             107.0  45.643836       0   
99881      172327  214377             101.0  77.816438       0   
99883      150755  276467             103.0  73.931507       0   
99893      128349  208873              94.0  88.608219       0   
99897      162913  266801             100.0  53.586301       0   
           181057  298720             101.0  54.493151       0   
99899      188409  245756              92.0  87.013699       0   
99901      131711  226541              86.0  68.873973       0   
99903      135939  291066             109.0  50.142466       0   
99923      164914  202505              92.0  54.424658       0   
           192053  298857             108.0  54.646575       0   
99928      104802  236704             104.0  20.884932       1   
99936      107913  213906             107.0  91.400000       1   
99937      129380  291004             103.0  65.720548       1   
99938      103815  296707             107.0  80.890411       0   
99939      159023  252442             107.0  26.967123       0   
99944      185654  221067             106.0  85.536986       1   
99946      157197  298613             105.0  49.698630       0   
99957      148574  257127              98.0  68.095890       0   
99965      101083  257338             108.0  66.334247       0   
99966      167228  252173             106.0  77.536986       0   
99982      112748  275958              99.0  65.304110       0   
           151454  221194             100.0  65.200000       0   
           183791  239827              92.0  65.419178       0   
99983      117390  286606             106.0  78.627397       0   
99985      176670  279638             105.0  53.843836       0   
99992      197084  242052             104.0  65.813699       1   
99995

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train, test = train_test_split(df, train_size=0.7, random_state=60611)

train

/Users/panyiheng/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


,,,chloride_max,age,gender,chloride_input_meq,fluid_net_input_ml,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,...,sysbp_min,diasbp_min,resprate_max,weight,mingcs,norepinephrine,ASIAN,BLACK,HISPANIC,WHITE
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
8050,198956,273342,106.0,81.964384,0,87.010000,1691.900000,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,0
697,162458,231583,109.0,26.616438,0,368.065000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,1
29093,196672,203201,108.0,84.767123,1,12.320000,1165.000000,0.0,0.0,0.0,0.0,0.0,...,99.0,41.0,34.0,53.099998,14.0,0.0,1,0,0,0
55886,180377,265100,107.0,91.400000,0,60.264106,-1415.594788,1.0,0.0,0.0,1.0,0.0,...,87.0,36.0,27.0,78.500000,14.0,0.0,0,0,0,1
18809,124161,277454,103.0,83.054795,1,283.360000,-135.000000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,1
13922,198138,217191,106.0,52.487671,0,812.934267,0.000000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,1
79873,103496,274385,89.0,91.400000,1,263.346702,-2315.670766,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,0,0
88361,108180,265768,106.0,76.257534,0,369.599991,899.999941,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,1
91513,181783,295396,101.0,56.706849,1,251.071333,1043.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,1


In [13]:
train.to_csv('train_data.csv')
test.to_csv('test_data.csv')

In [18]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [33]:
stan =pd.read_csv('train_standard.csv')

In [34]:
stan = stan.set_index(['vitalid'])
stan

,mean,median,std,iqr
vitalid,,,,
BICARBONATE,26.353636,26.000000,4.923168,6.000000
CHLORIDE,102.274132,103.000000,5.365228,7.000000
SODIUM,137.808282,138.000000,4.833594,6.000000
DiasBP,59.454354,58.000000,14.793751,19.000000
HeartRate,82.593139,81.000000,15.954386,22.000000
RespRate,19.601587,19.000000,5.353180,7.000000
SysBP,123.090283,121.000000,23.119897,32.000000
mingcs,13.836619,2.140535,15.000000,1.000000
fluid_net_input_ml,821.883052,707.229561,2453.556938,1847.313465


In [35]:
stan.loc['HeartRate']['median']

81.0

# Normalization

## impute with median

In [37]:
train =train.replace({'heartrate_max': {0: stan.loc['HeartRate']['median']},
               'sysbp_min': {0: stan.loc['SysBP']['median']},
               'diasbp_min': {0: stan.loc['DiasBP']['median']},
               'resprate_max': {0: stan.loc['RespRate']['median']},
               'weight': {0: stan.loc['weight']['median']},
               'mingcs': {0: stan.loc['mingcs']['median']},
               'age': {0: stan.loc['age']['median']},
               'sodium_max': {0: stan.loc['SODIUM']['median']},
               'bicarbonate_min': {0: stan.loc['BICARBONATE']['median']},
               'fluid_net_input_ml': {0: stan.loc['fluid_net_input_ml']['median']},
              }) 


,subject_id,hadm_id,icustay_id,chloride_max,age,gender,chloride_input_meq,fluid_net_input_ml,congestive_heart_failure,cardiac_arrhythmias,...,sysbp_min,diasbp_min,resprate_max,weight,mingcs,norepinephrine,ASIAN,BLACK,HISPANIC,WHITE
0,8050,198956,273342,106.0,81.964384,0,87.010000,1691.900000,1.0,1.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,0,0
1,697,162458,231583,109.0,26.616438,0,368.065000,707.229561,0.0,0.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,0,1
2,29093,196672,203201,108.0,84.767123,1,12.320000,1165.000000,0.0,0.0,...,99.0,41.0,34.0,53.099998,14.000000,0.0,1,0,0,0
3,55886,180377,265100,107.0,91.400000,0,60.264106,-1415.594788,1.0,0.0,...,87.0,36.0,27.0,78.500000,14.000000,0.0,0,0,0,1
4,18809,124161,277454,103.0,83.054795,1,283.360000,-135.000000,0.0,1.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,0,1


In [38]:
test =test.replace({'heartrate_max': {0: stan.loc['HeartRate']['median']},
               'sysbp_min': {0: stan.loc['SysBP']['median']},
               'diasbp_min': {0: stan.loc['DiasBP']['median']},
               'resprate_max': {0: stan.loc['RespRate']['median']},
               'weight': {0: stan.loc['weight']['median']},
               'mingcs': {0: stan.loc['mingcs']['median']},
               'age': {0: stan.loc['age']['median']},
               'sodium_max': {0: stan.loc['SODIUM']['median']},
               'bicarbonate_min': {0: stan.loc['BICARBONATE']['median']},
               'fluid_net_input_ml': {0: stan.loc['fluid_net_input_ml']['median']},
              }) 
test

,subject_id,hadm_id,icustay_id,chloride_max,age,gender,chloride_input_meq,fluid_net_input_ml,congestive_heart_failure,cardiac_arrhythmias,...,sysbp_min,diasbp_min,resprate_max,weight,mingcs,norepinephrine,ASIAN,BLACK,HISPANIC,WHITE
0,12103,112106,240697,107.0,56.172603,0,189.662500,707.229561,0.0,0.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,0,1
1,12706,134578,219408,107.0,61.479452,1,185.481000,1047.000000,1.0,0.0,...,110.0,29.0,25.0,44.900000,14.000000,0.0,0,0,0,1
2,57186,103276,287690,108.0,37.558904,1,459.313771,707.229561,0.0,0.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,0,1
3,10046,133110,213289,100.0,53.400000,1,194.880000,707.229561,0.0,0.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,0,1
4,28219,127319,283735,107.0,77.668493,1,121.275000,1404.578129,0.0,1.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,0,1
5,14408,141316,294204,108.0,72.657534,1,350.400000,2088.904005,0.0,1.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,0,1
6,18614,109642,214323,106.0,83.600000,1,42.350000,1080.000000,0.0,1.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,0,1
7,65991,137256,247035,107.0,31.136986,0,655.513526,707.229561,0.0,0.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,1,0
8,18928,161397,263520,109.0,73.227397,0,80.000000,707.229561,0.0,0.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,0,1
9,26560,169774,255271,104.0,29.306849,0,0.000000,-890.000000,0.0,0.0,...,121.0,58.0,19.0,78.450001,2.140535,0.0,0,0,1,0


In [39]:
train.to_csv('train.csv')
test.to_csv('test.csv')

## normalize

In [92]:
df1['heartrate_max'] = (df1['heartrate_max'] - stan.loc['HeartRate']['median'])/stan.loc['HeartRate']['iqr']
df1['sysbp_min'] = (df1['sysbp_min'] - stan.loc['SysBP']['median'])/stan.loc['SysBP']['iqr']
df1['diasbp_min'] = (df1['diasbp_min'] - stan.loc['DiasBP']['median'])/stan.loc['DiasBP']['iqr']
df1['resprate_max'] = (df1['resprate_max'] - stan.loc['RespRate']['median'])/stan.loc['RespRate']['iqr']
df1['weight'] = (df1['weight'] - df_wt['weight'].median())/(df_wt['weight'].quantile(0.75) - df_wt['weight'].quantile(0.25))
df1['mingcs'] = (df1['mingcs'] - stan.loc['mingcs']['median'])/stan.loc['mingcs']['iqr']

df1['chloride_max'] = (df1['chloride_max'] - stan.loc['CHLORIDE']['median'])/stan.loc['CHLORIDE']['iqr']
df1['sodium_max'] = (df1['sodium_max'] - stan.loc['SODIUM']['median'])/stan.loc['SODIUM']['iqr']
df1['bicarbonate_min'] = (df1['bicarbonate_min'] - stan.loc['BICARBONATE']['median'])/stan.loc['BICARBONATE']['iqr']

In [94]:
print(df1['age'].median(), (df1['age'].quantile(0.75) - df1['age'].quantile(0.25)),df1['age'].mean(),df1['age'].std() )

65.7479452054795 24.945205479452092 64.15323589540823 17.215711139799687


In [95]:
print(df_wt['weight'].median(), (df_wt['weight'].quantile(0.75) - df_wt['weight'].quantile(0.25)),df_wt['weight'].mean(),df_wt['weight'].std() )

80.0 28.0 83.06121690261101 24.71410967312458


In [96]:
df1['fluid_net_input_ml'] = (df1['fluid_net_input_ml'] - df1['fluid_net_input_ml'].median())/ (df1['fluid_net_input_ml'].quantile(0.75) - df1['fluid_net_input_ml'].quantile(0.25))
df1['age'] = (df1['age'] - df1['age'].median())/ (df1['age'].quantile(0.75) - df1['age'].quantile(0.25))

df1

chloride_max       age  gender  \
subject_id hadm_id icustay_id                                   
3          145834  211552             1.250  0.434047       0   
4          185777  294638             0.500 -0.716420       1   
6          107064  228232            -0.375  0.009445       1   
9          150750  220597            -0.125 -0.959363       0   
11         194540  229441             0.250 -0.624053       1   
12         112213  232669             1.125  0.267435       0   
13         143045  263738             1.500 -1.036573       1   
17         161087  257980             1.125 -0.717518       1   
           194023  277042             1.000 -0.732125       1   
18         188822  298129            -0.250 -0.596266       0   
19         109235  273430             0.125  1.028336       0   
20         157681  264490             0.500  0.408018       1   
21         109451  217847             0.000  0.871829       0   
           111970  216859             0.000  0.887315       0   
22         165315  204798             0.125 -0.031192       1   
23         124321  234044             0.500  0.383086       0   
           152223  227807             1.250  0.217683       0   
24         161859  262236             0.375 -1.070621       0   
25         129635  203487             0.875 -0.270950       0   
26         197661  244882             0.250  0.252718       0   
28         162569  225559             1.000  0.348160       0   
30         104557  225176            -0.875  1.028336       0   
31         128652  254478            -0.125  0.263152       0   
32         175413  295037             0.000 -0.809995       0   
33         176176  296681             0.250  0.669412       0   
34         115799  263086             0.125  1.028336       0   
           144319  290505             0.375  1.028336       0   
35         166707  282039             2.000  0.149918       1   
36         122659  211200             0.625  0.161560       0   
           165660  241249             0.625  0.281713       0   
...                                     ...       ...     ...   
99897      181057  298720            -0.375 -0.451181       0   
99899      188409  245756            -1.500  0.852499       0   
99901      131711  226541            -2.250  0.125316       0   
99903      135939  291066             0.625 -0.625590       0   
99912      189380  236183             1.000  0.755739       0   
99913      175989  292375             1.000  0.216035       0   
99922      123563  259919             1.000  0.143438       0   
99923      164914  202505            -1.500 -0.453926       0   
           192053  298857             0.500 -0.445030       0   
99928      104802  236704             0.000 -1.798462       1   
99936      107913  213906             0.375  1.028336       1   
99937      129380  291004            -0.125 -0.001098       1   
99938      103815  296707             0.375  0.607029       0   
99939      159023  252442             0.375 -1.554640       0   
99944      185654  221067             0.250  0.793300       1   
99946      157197  298613             0.125 -0.643383       0   
99955      108494  243255             1.375 -0.100824       0   
99957      148574  257127            -0.750  0.094124       0   
99965      101083  257338             0.500  0.023504       0   
99966      167228  252173             0.250  0.472597       0   
99973      150202  275083             1.625 -0.008237       1   
99982      112748  275958            -0.625 -0.017792       0   
           151454  221194            -0.500 -0.021966       0   
           183791  239827            -1.500 -0.013180       0   
99983      117390  286606             0.250  0.516310       0   
99985      176670  279638             0.125 -0.477210       0   
99991      151118  226241             1.250 -0.721582       0   
99992      197084  242052             0.000  0.002636       1   
99995      137810  229633            -0.500  0.922350       1   
9

In [97]:
df1.to_csv('data_normalize_median.csv')